In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
# Optionally import seaborn for nicer plots
import seaborn as sns

In [ ]:
# Assuming you are in the project root or adjust path accordingly
# from src.config_transfer import MODELS_DIR, HISTORY_NAME, MODEL_NAME
# If not importing config_transfer, manually define paths:
MODELS_DIR = 'models' # Or the full path if running from outside project root
HISTORY_NAME = 'training_history_transfer.csv'
MODEL_NAME = 'ra_classifier_transfer_cnn.h5' # For confusion matrix

history_file_path = os.path.join(MODELS_DIR, HISTORY_NAME)
model_file_path = os.path.join(MODELS_DIR, MODEL_NAME)

In [ ]:
# Load the training history from the CSV file
history_df = pd.read_csv(history_file_path)
print(history_df.head()) # See the first few rows

In [ ]:
plt.figure(figsize=(15, 10))

# Plot Loss
plt.subplot(2, 2, 1)
plt.plot(history_df['loss'], label='Training Loss')
plt.plot(history_df['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot Accuracy
plt.subplot(2, 2, 2)
plt.plot(history_df['accuracy'], label='Training Accuracy')
plt.plot(history_df['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot Precision and Recall
plt.subplot(2, 2, 3)
plt.plot(history_df['precision'], label='Training Precision')
plt.plot(history_df['recall'], label='Training Recall')
plt.plot(history_df['val_precision'], label='Validation Precision')
plt.plot(history_df['val_recall'], label='Validation Recall')
plt.title('Precision & Recall over Epochs (for RA)')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.legend()
plt.grid(True)

# Plot AUC-ROC
plt.subplot(2, 2, 4)
plt.plot(history_df['auc_roc'], label='Training AUC-ROC')
plt.plot(history_df['val_auc_roc'], label='Validation AUC-ROC')
plt.title('AUC-ROC over Epochs')
plt.xlabel('Epoch')
plt.ylabel('AUC Score')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show() # Display the plots
# plt.savefig(os.path.join(MODELS_DIR, 'training_curves.png')) # Save the plots

In [ ]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, recall_score, precision_score
import seaborn as sns # For heatmap

# Assuming you have test_generator from a data_loader run
# You might need to rerun data_loader_transfer.py to get test_generator
# from src.data_loader_transfer import get_image_data_generators
# _, _, test_generator = get_image_data_generators()

# Load the best model
# best_model = tf.keras.models.load_model(model_file_path)

# Make predictions (y_pred_probs are probabilities from 0 to 1)
# y_pred_probs = best_model.predict(test_generator)
# y_pred_binary = (y_pred_probs > 0.5).astype(int) # Apply threshold

# Get true labels (assuming test_generator.classes is available or you manually load labels)
# test_generator.reset() # Reset if it's already been iterated
# y_true = test_generator.classes

# Plot Confusion Matrix
# cm = confusion_matrix(y_true, y_pred_binary)
# plt.figure(figsize=(6, 5))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
#             xticklabels=['Healthy', 'RA'], yticklabels=['Healthy', 'RA'])
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.title('Confusion Matrix (Test Set)')
# plt.show()

# (You already calculate F1-Score and Specificity in train_transfer.py,
# so you could just display those from the console output or store them in a summary file.)